In [207]:
from Bio import SeqIO
import pandas as pd

def extractHLAfromFasta(filename):
    """ ดึงข้อมูล HLA ออกจาก fasta file แล้ว return list of HLA Alleles """
    HLAnomenclature_list = []
    for seq_record in SeqIO.parse(filename, "fasta"):  
        seq_description_list = seq_record.description.split()
        HLAnomenclature = seq_description_list[1]
        HLAnomenclature_list.append(HLAnomenclature)
    return HLAnomenclature_list

def extractGenefromHLA(listofHLA):
    """ รับ list of HLA alleles, แล้ว return list of Gene"""
    gene_list = []
    for HLA in listofHLA:
        gene = HLA.split("*")[0]
        gene_list.append(gene)
    return gene_list

def returnuniquefromlist(list):
    """ รับ list ใด ๆ แล้ว return unique values of the list"""
    unique_list = []
    loaded_list = pd.DataFrame(list).drop_duplicates()
    for lab,row in loaded_list.iterrows():
        unique_list.append(row.item())
    return unique_list

def extractHLAfromGenelist(listofHLA, listofgene):
    """ รับ list of gene แล้ว return list of HLA ที่มียีนนั้น """
    return_list = []
    for HLA in listofHLA:
        gene = HLA.split("*")[0]
        if gene in listofgene:
            return_list.append(HLA)
        else:
            continue
    return return_list

In [208]:
# ดึงข้อมูลจาก fasta
fasta_HLA = extractHLAfromFasta('hla_prot.fasta')
fasta_gene = extractGenefromHLA(fasta_HLA)
fasta_unique_gene = returnuniquefromlist(fasta_gene)
fasta_unique_HLA = returnuniquefromlist(fasta_HLA)

In [209]:
#ดึงข้อมูลจาก datasets
file = open('../3 - Cleaning & Transforming Data/log/log_notclean_Allele.txt', 'r')
dataset_HLA = []
for x in file.readlines():
    x = x.split()
    allele = x[0]
    dataset_HLA.append(allele)
dataset_HLA.sort()
dataset_HLA = list(dataset_HLA[1:])
file.close()

dataset_gene = extractGenefromHLA(dataset_HLA)
dataset_unique_HLA = returnuniquefromlist(dataset_HLA)
dataset_unique_gene = returnuniquefromlist(dataset_gene)

In [210]:
print(f"ในฐานข้อมูล fasta จะมียีนทั้งหมด {len(fasta_unique_gene)} แต่ใน dataset เราจะมี {len(dataset_unique_gene)}")

ในฐานข้อมูล fasta จะมียีนทั้งหมด 26 แต่ใน dataset เราจะมี 13


In [211]:
#หาว่า gene อะไรที่มีทั้งใน dataset และ fasta (จะได้ทำ alignment ของยีนพวกนี้)
final_gene = list(set(fasta_unique_gene) & set(dataset_unique_gene))
final_gene.sort()
final_classone_gene, final_classtwo_gene = final_gene[0:3], final_gene[3:]
print(final_classone_gene, final_classtwo_gene)

['A', 'B', 'C'] ['DPB1', 'DQA1', 'DQB1', 'DRB1', 'DRB3', 'DRB4', 'DRB5']


In [212]:
# ดึงเอาแต่ข้อมูล HLA ที่ตรงกับ gene list ที่ใส่เข้าไป
# จะได้ข้อมูล Alleles ที่จะนำมาทำ Alignments
class1HLAtoallignment = extractHLAfromGenelist(fasta_HLA, final_classone_gene)
class2HLAtoallignment = extractHLAfromGenelist(fasta_HLA, final_classtwo_gene)

In [213]:
def createnewfastafromHLAlist_separatedgene(filename, HLAtoalignment, outputname):
    """ สร้าง HLA ใหม่โดยใช้ HLA list"""
    new_list = []
    for seq_record in SeqIO.parse(filename, "fasta"):  
        seq_description_list = seq_record.description.split()
        HLAnomenclature = seq_description_list[1]
        if HLAnomenclature in HLAtoalignment:
            new_list.append(seq_record)
    return SeqIO.write(new_list, outputname, "fasta")

def createnewfastafromGenelist(fastafile, ListofGeneWantedinfile):
    """ สร้าง fasta ใหม่โดยใช้ list gene ที่ต้องการในไฟล์นั้น ๆ """
    new_list = []
    for gene in ListofGeneWantedinfile:
        print(f"Processing for gene: {gene}")
        for seq_record in SeqIO.parse(fastafile, "fasta"):  
            seq_description_list = seq_record.description.split()
            HLAnomenclature = seq_description_list[1]
            GenefromHLAnomenclature = HLAnomenclature.split('*')[0]
            if GenefromHLAnomenclature == gene:
                new_list.append(seq_record)
    SeqIO.write(new_list, f'gene_aligned_{ListofGeneWantedinfile}.fasta', "fasta")
    print(f"Saved for gene: {ListofGeneWantedinfile}")
    print("All Done")

def dropuniqueHLAvalue(fastafile, fieldwanted,dropchars):
    """ ลบ HLA ที่ซ้ำกันออกจาก fasta file โดย field wanted = 1 ถึง 4"""
    new_df = pd.DataFrame(columns=['HLA','field1','field2','field3','field4'])
    seq_record_list = []
    
    total_counts = 0
    for seq_record in SeqIO.parse(fastafile, "fasta"):  
        total_counts += 1
        seq_description_list = seq_record.description.split()
        HLAnomenclature = seq_description_list[1]
        GenefromHLAnomenclature = HLAnomenclature.split('*')[1]
        HLAnomenclature_splited = GenefromHLAnomenclature.split(':')

        #Add - if field is empty
        while len(HLAnomenclature_splited) < 4:
            HLAnomenclature_splited.append('-')

        to_add = pd.DataFrame({'HLA': HLAnomenclature,
        'field1':HLAnomenclature_splited[0],
        'field2':HLAnomenclature_splited[1],
        'field3':HLAnomenclature_splited[2],
        'field4':HLAnomenclature_splited[3]}, index=[0])

        new_df = new_df.append(to_add, ignore_index=True)
    new_df = new_df.reset_index(drop=True)
    df_dropped = new_df.drop_duplicates(subset=[f'field{x+1}' for x in range(fieldwanted)], keep='first')

    # Process drop chars
    if dropchars == True:
        to_drop = []
        #counter = 0
        for x in df_dropped.iterrows():
            index = -1
            y = x[-1][index][-1]
            while(y == '-'):
                index -= 1
                y = x[-1][index][-1]
            if y in ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']:
                to_drop.append(x[0])
            #counter += 1
        df_dropped = df_dropped.drop(to_drop)

    # Save file
    index_counter = 0
    for seq_record in SeqIO.parse(fastafile, "fasta"):  
        if index_counter in df_dropped.index:
            seq_record_list.append(seq_record)
        index_counter += 1

    # print(new_df)
    # print(df_dropped)
    # print(df_dropped.index)
    # # print(new_list)
    # print(seq_record_list)
    # f = open(f'{fastafile}_dropdupe{fieldwanted}field_dropchar{str(dropchars)}.txt', "w")
    # f.write()
    # f.close()
    SeqIO.write(seq_record_list, f'{fastafile}_dropdupe{fieldwanted}field_dropchar{str(dropchars)}.fasta', "fasta")
    
    #print(f"Saved for gene: {ListofGeneWantedinfile}")
    print(f"{total_counts} were removed to {len(seq_record_list)} records")
    print("All Done")

#สร้าง csv จาก gene list
def createnewfastafromGenelist_separatedgene_csv(filename, Genetoallignment):
    for gene in Genetoallignment:
        new_csv = pd.DataFrame(columns=['Alleles','Sequence'])
        print(f"Processing for gene: {gene}")
        for seq_record in SeqIO.parse(filename, "fasta"):  
            seq_description_list = seq_record.description.split()
            HLAnomenclature = seq_description_list[1]
            GenefromHLAnomenclature = HLAnomenclature.split('*')[0]
            allele = seq_record.description.split()[1]
            sequence = repr(seq_record.seq).split("'")[1]
            if GenefromHLAnomenclature == gene:
                new_csv = new_csv.append({"Alleles":allele,"Sequence":sequence},ignore_index=True)
        print(f"Saved for gene: {gene}")
        new_csv.to_csv(f'gene_{gene}.csv')
    print("All Done")

# แปลง fasta เป็น csv
def convertfastatocsv(filename):
    """ Input file in fasta format to convert to csv """
    new_csv = pd.DataFrame(columns=['Alleles','Sequence'])
    for seq_record in SeqIO.parse(filename, "fasta"):  
        seq_description_list = seq_record.description.split()
        HLAnomenclature = seq_description_list[1]
        GenefromHLAnomenclature = HLAnomenclature.split('*')[0]
        allele = seq_record.description.split()[1]
        sequence = ""
        for x in list(seq_record.seq):
            sequence = sequence+x
        new_csv = new_csv.append({"Alleles":allele,"Sequence":sequence,"Char":len(sequence)},ignore_index=True)
    new_csv.to_csv(f'gene_{GenefromHLAnomenclature}_aligned.csv',index=False)

# def convertfastatocsv_unalligned(filename):
#     """ Input file in fasta format to convert to csv """
#     new_csv = pd.DataFrame(columns=['Alleles','Sequence'])
#     for seq_record in SeqIO.parse(filename, "fasta"):  
#         seq_description_list = seq_record.description.split()
#         HLAnomenclature = seq_description_list[1]
#         GenefromHLAnomenclature = HLAnomenclature.split('*')[0]
#         allele = seq_record.description.split()[1]
#         sequence = ""
#         for x in list(seq_record.seq):
#             sequence = sequence+x
#         new_csv = new_csv.append({"Alleles":allele,"Sequence":sequence,"Char":len(sequence)},ignore_index=True)
#     new_csv.to_csv(f'gene_{GenefromHLAnomenclature}_hla_prot.csv',index=False)


In [216]:
createnewfastafromGenelist('hla_prot.fasta', final_classone_gene)
createnewfastafromGenelist('hla_prot.fasta', final_classtwo_gene)

Processing for gene: A
Processing for gene: B
Processing for gene: C
Saved for gene: ['A', 'B', 'C']
All Done
Processing for gene: DPB1
Processing for gene: DQA1
Processing for gene: DQB1
Processing for gene: DRB1
Processing for gene: DRB3
Processing for gene: DRB4
Processing for gene: DRB5
Saved for gene: ['DPB1', 'DQA1', 'DQB1', 'DRB1', 'DRB3', 'DRB4', 'DRB5']
All Done


In [205]:
dropuniqueHLAvalue("gene_aligned_['A', 'B', 'C'].fasta", 2, dropchars=True)

24543 were removed to 12370 records
All Done


In [204]:
dropuniqueHLAvalue("gene_aligned_['DPB1', 'DQA1', 'DQB1', 'DRB1', 'DRB3', 'DRB4', 'DRB5'].fasta", 2, dropchars=True)

9633 were removed to 4787 records
All Done


# ข้างล่างนี้ไม่เกี่ยวละ

In [ ]:
# createnewfastafromGenelist('hla_prot.fasta', final_classone_gene)
# createnewfastafromGenelist('hla_prot.fasta', final_classtwo_gene)

In [16]:
#batch covert all aligned fasta file to csv
for x in final_gene:
    convertfastatocsv(f'gene_{x}_hla_prot_aligned.fasta')

In [ ]:
#batch covert all UNALLIGNED fasta file to csv
#เพื่อ select ว่าอันไหนน่าจะเป้นตัวแทนที่ดีที่สุดในการเลือก
for x in final_gene:
    convertfastatocsv_unalligned(f'gene_{x}_hla_prot.fasta')
    df = pd.read_csv(f'gene_{x}_hla_prot.csv')
    try:
        max = df[df['Char'] == df['Char'].max()].values[[0,1],[0,2]]
        min = df[df['Char'] == df['Char'].min()].values[[0,1],[0,2]]
    except:
        max = df[df['Char'] == df['Char'].max()].values[0][[0,2]]
        min = df[df['Char'] == df['Char'].min()].values[0][[0,2]]
    print(max)
    print(min)
    print("with average of " + str(df['Char'].mean()))
    print("---------------------------------")

In [ ]:
def returnHLAtwofirstfield(HLAlist):
    return_list = []
    for HLA in HLAlist:
        splitedHLA = HLA.split(":")
        HLAtwo = splitedHLA[0:2]
        newHLA = ':'.join(HLAtwo)
        return_list.append(newHLA)
    return return_list

In [227]:
#HLA 2 field แรกจากฐานข้อมูล fasta
returnuniquefromlist(returnHLAtwofirstfield(fasta_HLA))

['A*01:01',
 'A*01:02',
 'A*01:03',
 'A*01:04',
 'A*01:06',
 'A*01:07',
 'A*01:08',
 'A*01:09',
 'A*01:10',
 'A*01:11N',
 'A*01:12',
 'A*01:13',
 'A*01:14',
 'A*01:15N',
 'A*01:16N',
 'A*01:17',
 'A*01:18N',
 'A*01:19',
 'A*01:20',
 'A*01:21',
 'A*01:22N',
 'A*01:23',
 'A*01:24',
 'A*01:25',
 'A*01:26',
 'A*01:27N',
 'A*01:28',
 'A*01:29',
 'A*01:30',
 'A*01:31N',
 'A*01:32',
 'A*01:33',
 'A*01:35',
 'A*01:36',
 'A*01:37',
 'A*01:38',
 'A*01:39',
 'A*01:40',
 'A*01:41',
 'A*01:42',
 'A*01:43',
 'A*01:44',
 'A*01:45',
 'A*01:46',
 'A*01:47',
 'A*01:48',
 'A*01:49',
 'A*01:50',
 'A*01:51',
 'A*01:52',
 'A*01:53N',
 'A*01:54',
 'A*01:55',
 'A*01:56N',
 'A*01:57N',
 'A*01:58',
 'A*01:59',
 'A*01:60',
 'A*01:61',
 'A*01:62',
 'A*01:63',
 'A*01:64',
 'A*01:65',
 'A*01:66',
 'A*01:67',
 'A*01:68',
 'A*01:69',
 'A*01:70',
 'A*01:71',
 'A*01:72',
 'A*01:73',
 'A*01:74',
 'A*01:75',
 'A*01:76',
 'A*01:77',
 'A*01:78',
 'A*01:79',
 'A*01:80',
 'A*01:81',
 'A*01:82',
 'A*01:83',
 'A*01:84',
 'A*01